To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

Use our [Llama-3 8b Instruct](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing) notebook for conversational style finetunes.

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
import sys
from pathlib import Path

workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("workding dir:", workding_dir)

workding dir: /home/inflaton/code/projects/courses/novel-translation


In [0]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /home/inflaton/code/projects/courses/novel-translation/.env


True

In [0]:
import os

model_name = os.getenv("MODEL_NAME")
token = os.getenv("HF_TOKEN") or None
load_in_4bit = os.getenv("LOAD_IN_4BIT") == "true"
local_model = os.getenv("LOCAL_MODEL")
hub_model = os.getenv("HUB_MODEL")
num_train_epochs = int(os.getenv("NUM_TRAIN_EPOCHS") or 0)
data_path = os.getenv("DATA_PATH")

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)


model_name, load_in_4bit, max_seq_length, num_train_epochs, dtype, data_path

('unsloth/Qwen2-0.5B-Instruct', True, 2048, 10, None, 'datasets/mac/mac.tsv')

* We support Llama, Mistral, Phi-3, Gemma, Yi, DeepSeek, Qwen, TinyLlama, Vicuna, Open Hermes etc
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

In [0]:
%%time

from translation_engine import *

model, tokenizer = load_model(model_name, load_in_4bit)

==((====))==  Unsloth: Fast Qwen2 patching release 2024.5
   \\   /|    GPU: NVIDIA GeForce RTX 4080 Laptop GPU. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 7.05 s, sys: 1.85 s, total: 8.89 s
Wall time: 33.4 s


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [0]:
from translation_engine import *

datasets = load_translation_dataset(data_path, tokenizer=tokenizer)

loading train/test data files


Map:   0%|          | 0/4528 [00:00<?, ? examples/s]

Map:   0%|          | 0/1133 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 4528
    })
    test: Dataset({
        features: ['chinese', 'english', 'text', 'prompt'],
        num_rows: 1133
    })
})


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [0]:
%%time

prompt1 = datasets["test"]["prompt"][0]
print(prompt1)
print("--" * 20)
test_model(model, tokenizer, prompt1)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant

----------------------------------------
<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant
Old Tong holds the gun, his eyes narrowed as he pulls out a triangular eyepiece and fires it. A rain of bullets crashes down like hailstones on the tree trunks, making a sound that echoes through the air.<|im_end|>
CPU times: user 1.94 s, sys: 99.2 ms, total: 2.04 s
Wall time: 2.02 s


In [0]:
print(datasets["test"]["english"][0])

Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.


In [0]:
%%time

wrong_prompt1 = datasets["test"]["text"][0]
print(wrong_prompt1)
print("--" * 20)
test_model(model, tokenizer, wrong_prompt1)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant
Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.<|im_end|>
----------------------------------------
<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
老耿端起枪，眯缝起一只三角眼，一搂扳机响了枪，冰雹般的金麻雀劈哩啪啦往下落，铁砂子在柳枝间飞迸着，嚓嚓有声。<|im_end|>
<|im_start|>assistant
Old Geng picked up his shotgun, squinted, and pulled the trigger. Two sparrows crashed to the ground like hailstones as shotgun pellets tore noisily through the branches.<|im_end|>
<|endoftext|>
CPU times: user 232 ms, sys: 0 ns, total: 232 ms
Wall time: 230 ms


In [0]:
%%time

prompt2 = datasets["test"]["prompt"][1]

test_model(model, tokenizer, prompt2)

<|im_start|>system
You are an expert in translating Chinese into English.<|im_end|>
<|im_start|>user
Translate from Chinese to English.
次日天未明时，刘老老便起来梳洗了， 又将板儿教了几句话； 五六岁的孩子，听见带了他进城逛去，喜欢的无不应承。<|im_end|>
<|im_start|>assistant
The next morning, when the sun was not yet up, Liu Tao woke up and got ready for his day. He also taught his son a few words; children five and six years old heard him going out with them to the city and were very happy to accept it.<|im_end|>
CPU times: user 2.17 s, sys: 136 ms, total: 2.3 s
Wall time: 2.28 s


In [0]:
print(datasets["test"]["english"][1])

Next day Grannie Liu was up before dawn. As soon as she had washed and done her hair, she set about teaching Ban-er a few words to say to the ladies at the great house – an exercise to which he submitted cheerfully enough, as would any little boy of four or five who had been promised an outing to the great city.


In [0]:
%%time

predictions = eval_model(model, tokenizer, datasets["test"])

100%|██████████| 1133/1133 [22:32<00:00,  1.19s/it]   

CPU times: user 20min 56s, sys: 1min 36s, total: 22min 33s
Wall time: 22min 32s


In [0]:
calc_metrics(datasets["test"]["english"], predictions, debug=True)

{'accuracy': 0.00088261253309797,
 'correct_ids': [659],
 'bleu_scores': {'bleu': 0.052034054574319276,
  'precisions': [0.31365077331967084,
   0.07768474215842637,
   0.026901189860320744,
   0.01118399827938488],
  'brevity_penalty': 1.0,
  'length_ratio': 1.034415369327592,
  'translation_length': 31229,
  'reference_length': 30190},
 'rouge_scores': {'rouge1': 0.3216675111173276,
  'rouge2': 0.09758257854743044,
  'rougeL': 0.2657220893441391,
  'rougeLsum': 0.265983838226093}}

In [0]:
datasets["test"]["chinese"][659]

'一声枪响。'

In [0]:
print(datasets["test"]["english"][659])

A shot rang out.


In [0]:
predictions[659]

'A shot rang out.'

In [0]:
%%time

# Empty VRAM
del model

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

CPU times: user 232 ms, sys: 904 μs, total: 233 ms
Wall time: 260 ms


0

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>